# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/vanducng/git/NanoDegree-Project02


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION =
    {'class' : 'SimpleStrategy', 'replication_factor': 1};
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace("sparkify")
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# As specified on query#1, we need to create a table including all required columns (session_id, item_in_session, etc.)
# The primary key is derived from fileter columns and additional columns if needed to uniquely identity a single row
# Here we choose the primary key is a combination of session_id and item_in_session

query = "CREATE TABLE IF NOT EXISTS music_log_by_session "
query += "(session_id int, item_in_session int, artist text, song_title text, song_length float, PRIMARY KEY(session_id, item_in_session));"
session.execute(query);

In [9]:
# Iterate the event_datafile_new.csv and insert respective columns into music_log_by_session table
# When implement this task in etl.py script, we can directly get the list of even_data via a variable 
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_log_by_session(session_id, item_in_session, artist, song_title, song_length)"
        query = query + " VALUES(%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## This step is to verify the query result to fileter the music log with session_id = 338 and item_in_session = 4
## To present a more intutive data, the cassandra rows can be converted to dataframe and display as tabular form
## Details has been implemented on test.py script
try:
    rows = session.execute("SELECT * FROM music_log_by_session WHERE session_id = 338 AND item_in_session = 4;")
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(session_id=338, item_in_session=4, artist='Faithless', song_length=495.30731201171875, song_title='Music Matters (Mark Knight Dub)')


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
## Apply similar steps for 2nd table modeling by answering the question
## Give me the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
## As the required of ordering the log by item_in_session, the additional CLUSTERING ORDER BY has been included in the creating table query

query = "CREATE TABLE IF NOT EXISTS music_log_by_session_and_user "
query += "(session_id int, user_id int, item_in_session int, artist text, song_title text, user_first_name text, user_last_name text, PRIMARY KEY ((session_id, user_id), item_in_session)) "
query += " WITH CLUSTERING ORDER BY (item_in_session ASC);"
session.execute(query)

# Insert into newly created table with data from 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_log_by_session_and_user(session_id, user_id, item_in_session, artist, song_title, user_first_name, user_last_name)"
        query = query + " VALUES(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[0], line[9], line[1], line[4]))

# Validate the query result
try:
    rows = session.execute("SELECT * FROM music_log_by_session_and_user WHERE user_id = 10 AND session_id = 182;")
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(session_id=182, user_id=10, item_in_session=0, artist='Down To The Bone', song_title="Keep On Keepin' On", user_first_name='Sylvie', user_last_name='Cruz')
Row(session_id=182, user_id=10, item_in_session=1, artist='Three Drives', song_title='Greece 2000', user_first_name='Sylvie', user_last_name='Cruz')
Row(session_id=182, user_id=10, item_in_session=2, artist='Sebastien Tellier', song_title='Kilometer', user_first_name='Sylvie', user_last_name='Cruz')
Row(session_id=182, user_id=10, item_in_session=3, artist='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', user_first_name='Sylvie', user_last_name='Cruz')


In [12]:
## Apply similar steps for 3rd table modeling by answering the question
## Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
## To uniquely identify a row, the user_log_by_song table uses the composite key of (song_title, artist)
## Assumption: artist and song combination is unique (it may happen in practice when 2 artist with same name and produce same song title))
query = "CREATE TABLE IF NOT EXISTS user_log_by_song "
query += "(song_title text, artist text, user_first_name text, user_last_name text, PRIMARY KEY(song_title, artist))"
session.execute(query)

# Insert into newly created table with data from 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_log_by_song(song_title, artist, user_first_name, user_last_name)"
        query = query + " VALUES(%s, %s, %s, %s)"
        session.execute(query, (line[9], line[0], line[1], line[4]))

# Validate the query result
try:
    rows = session.execute("SELECT * FROM user_log_by_song WHERE song_title = 'All Hands Against His Own';")
except Exception as e:
    print(e)

for row in rows:
    print(row)
                    

Row(song_title='All Hands Against His Own', artist='The Black Keys', user_first_name='Jacqueline', user_last_name='Lynch')


### Drop the tables before closing out the sessions

In [14]:
## TO-DO: Drop the table before closing out the sessions
session.execute("DROP TABLE music_log_by_session")
session.execute("DROP TABLE music_log_by_session_and_user")
session.execute("DROP TABLE user_log_by_song")

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()